In [1]:
import os
import pandas as pd
import email

In [2]:
directory=r'C:\Users\p_uli\Desktop\Columbia University\Cursos\Fall 22\Capstone\Data\maildir'

subset=['taylor-m','fischer-m','buy-r','ybarbo-p','geaccone-t','allen-p']

msg_id= list()
msg_txt= list()

for folder in subset:
    for roots, dirs, files in os.walk(directory+"\\"+folder):
        for file in files:
            with open(roots+'\\'+file) as f:
                x= f.readlines()
                x=''.join(x)
                msg_txt.append(x)
                msg_id.append(roots[roots.index(folder):] + '\\'+file)


In [3]:
data=pd.DataFrame(zip(msg_id,msg_txt),columns=['message_id','message_text'])
data.head()

,message_id,message_text
0,taylor-m\all_documents\1,Message-ID: <25895411.1075859874878.JavaMail.e...
1,taylor-m\all_documents\10,Message-ID: <23416266.1075859875079.JavaMail.e...
2,taylor-m\all_documents\100,Message-ID: <20401126.1075859877231.JavaMail.e...
3,taylor-m\all_documents\1000,Message-ID: <9557191.1075859900977.JavaMail.ev...
4,taylor-m\all_documents\1001,Message-ID: <2650318.1075859901001.JavaMail.ev...


In [4]:
#import pickle
## Helper functions
def get_text_from_email(msg):
    '''To get the content from email objects'''
    parts = []
    for part in msg.walk():
        if part.get_content_type() == 'text/plain':
            parts.append( part.get_payload() )
    return ''.join(parts)

def split_email_addresses(line):
    '''To separate multiple email addresses'''
    if line:
        addrs = line.split(',')
        addrs = list(map(lambda x: x.strip(), addrs))
    else:
        addrs = None
    return addrs

In [5]:
%%time
# Parse the emails into a list email objects
messages = list(map(email.message_from_string, data['message_text']))
data.drop(columns=['message_text'], inplace=True)

# Get fields from parsed email objects
#keys = messages[0].keys()
keys=['Message-ID', 'Date', 'From', 'To', 'Subject', 'Cc', 'Bcc','X-From', 'X-To', 'X-cc', 'X-bcc', 'X-Folder', 'X-Origin', 'X-FileName']

for key in keys:
    data[key] = [doc[key] for doc in messages]
    
# Parse content from emails
data['body'] = list(map(get_text_from_email, messages))
# Split multiple email addresses
data['From'] = data['From'].map(split_email_addresses)
data['To']   = data['To'].map(split_email_addresses)

# Extract the root of 'file' as 'user'
data['user'] = data['message_id'].map(lambda x:x.split('/')[0])
del messages

data.head(1)

CPU times: total: 3.05 s
Wall time: 3.07 s


,message_id,Message-ID,Date,From,To,Subject,Cc,Bcc,X-From,X-To,X-cc,X-bcc,X-Folder,X-Origin,X-FileName,body,user
0,taylor-m\all_documents\1,<25895411.1075859874878.JavaMail.evans@thyme>,"Fri, 30 Oct 1998 07:43:00 -0800 (PST)",[mark.taylor@enron.com],[marc.r.cutler@bankamerica.com],Re: Friday,None,None,Mark - ECT Legal Taylor,Marc.R.Cutler@BankAmerica.com,,,\Mark_Taylor _Dec_2000\Notes Folders\All docum...,Taylor-M,mtaylor.nsf,Wish we could go - but we're off to Ft. Lauder...,taylor-m\all_documents\1


In [6]:
#message_id, date, user_id, from, to, subject, cc, bcc, body

df=data[['message_id','Date','From','To','Subject','Cc','Bcc','body']]
df=df.rename(columns={"Date": "date", "From": "from",'To':'to','Subject':'subject','Cc':'cc','Bcc':'bcc'})
df.head()


,message_id,date,from,to,subject,cc,bcc,body
0,taylor-m\all_documents\1,"Fri, 30 Oct 1998 07:43:00 -0800 (PST)",[mark.taylor@enron.com],[marc.r.cutler@bankamerica.com],Re: Friday,None,None,Wish we could go - but we're off to Ft. Lauder...
1,taylor-m\all_documents\10,"Wed, 4 Nov 1998 09:21:00 -0800 (PST)",[mark.taylor@enron.com],[mtaylor587@aol.com],Golfing with Jesus,None,None,"Moses, Jesus and an old man are golfing. Moses..."
2,taylor-m\all_documents\100,"Tue, 12 Jan 1999 01:25:00 -0800 (PST)",[mark.taylor@enron.com],[janet.dobernecker@enron.com],Houston,tana.jones@enron.com,tana.jones@enron.com,---------------------- Forwarded by Mark - ECT...
3,taylor-m\all_documents\1000,"Thu, 30 Dec 1999 01:10:00 -0800 (PST)",[mark.taylor@enron.com],[raislerk@sullcrom.com],Re: EnronCredit.com,None,None,Friday at 10 looks good to us.\n\n\n\n\nRAISLE...
4,taylor-m\all_documents\1001,"Thu, 30 Dec 1999 01:17:00 -0800 (PST)",[mark.taylor@enron.com],"[<""mark_kopinski/intl/acim/americancentury""@am...",Happy New Year,None,None,Thanks for your Christmas letter. It was good...


In [7]:
df.to_csv(r'C:\Users\p_uli\Desktop\Columbia University\Cursos\Fall 22\Capstone\Data\subset.csv',index=False)
# import pickle

# with open(r'C:\Users\p_uli\Desktop\Columbia University\Cursos\Fall 22\Capstone\Data\Subset.pickle', 'wb') as handle:
#     pickle.dump(df, handle, protocol=pickle.HIGHEST_PROTOCOL)

    
# with open('filename.pickle', 'rb') as handle:
#     b = pickle.loa